In [30]:
import pandas as pd
import numpy as np
import os
import json
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm import tqdm
import signal
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gaussian_kde
import plotly.graph_objects as go
from kgcpy import lookupCZ
import google.generativeai as genai
import os
import pandas as pd
import numpy as np
import openai
import json
from tqdm import tqdm
import ast
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [12]:
df_gpt = pd.read_pickle('data_ieee_gpt.pkl')
df_gemini = pd.read_pickle('data_ieee_gemini.pkl')
df_gemini = df_gemini.rename(columns={"pv technology": "pv tech"})

In [13]:
# Find DOIs that are unique in each DataFrame
unique_gpt = df_gpt['doi'].value_counts()
unique_gemini = df_gemini['doi'].value_counts()

# Filter DataFrames to only unique DOIs
df_gpt_unique = df_gpt[df_gpt['doi'].isin(unique_gpt[unique_gpt == 1].index)]
df_gemini_unique = df_gemini[df_gemini['doi'].isin(unique_gemini[unique_gemini == 1].index)]


In [14]:
doi_counts = df_gpt['doi'].value_counts()

# Count occurrences
num_1 = (doi_counts == 1).sum()
num_2 = (doi_counts == 2).sum()
num_gt2 = (doi_counts > 2).sum()
total = len(doi_counts)

# Calculate percentages
pct_1 = num_1 / total * 100
pct_2 = num_2 / total * 100
pct_gt2 = num_gt2 / total * 100

# Print results
print(f"DOI appears exactly once: {num_1} ({pct_1:.2f}%)")
print(f"DOI appears exactly twice: {num_2} ({pct_2:.2f}%)")
print(f"DOI appears more than twice: {num_gt2} ({pct_gt2:.2f}%)")

DOI appears exactly once: 548 (65.47%)
DOI appears exactly twice: 127 (15.17%)
DOI appears more than twice: 162 (19.35%)


In [15]:
# Get the set of unique DOIs present in both DataFrames
shared_unique_dois = set(df_gpt_unique['doi']).intersection(df_gemini_unique['doi'])

# Filter both DataFrames to rows with these shared DOIs
df_gpt_final = df_gpt_unique[df_gpt_unique['doi'].isin(shared_unique_dois)]
df_gemini_final = df_gemini_unique[df_gemini_unique['doi'].isin(shared_unique_dois)]

In [16]:
def rough_post_process(df):
    # Define values to replace
    values_to_replace = ['not reported', 'null', 'nan', 'NULL', 'NaN', 'NONE', '', None]
    # Replace them with np.nan
    df = df.replace(values_to_replace, np.nan)

    df['rate'] = pd.to_numeric(df['annual power degradation rate in percent'], errors='coerce')
    df['rate_abs'] = np.abs(df['rate'])
    df['paper id'] = pd.to_numeric(df['paper id'], errors='coerce').astype('Int64')
    df['publish year'] = pd.to_numeric(df['publish year'], errors='coerce')
    df['number of measurements for degradation analysis'] = pd.to_numeric(df['number of measurements for degradation analysis'], errors='coerce')
    df['start year'] = pd.to_numeric(df['start year'], errors='coerce')
    df['end year'] = pd.to_numeric(df['end year'], errors='coerce')
    df['duration'] = pd.to_numeric(df['duration in years'], errors='coerce')
    df['number of pv modules in study'] = pd.to_numeric(df['number of pv modules in study'], errors='coerce')
    df['system capacity'] = (
        df['system capacity in watts']
        .astype(str)  # Ensure all values are strings for processing
        .str.replace(' W', '', regex=False)  # Remove 'W'
    )
    df['system capacity'] = pd.to_numeric(df['system capacity'], errors='coerce')
    inx_temp = df['duration'].isna() & df['start year'].notna() & df['end year'].notna()
    df.loc[inx_temp, 'duration'] = df['end year'][inx_temp]- df['start year'][inx_temp]

    standardized_mapping = {
    'poly-c-Si': 'poly-c-Si',
    'multi-c-Si': 'poly-c-Si',
    'mc-Si': 'poly-c-Si',
    'polycrystalline': 'poly-c-Si',
    'p-Si': 'poly-c-Si',

    'a-Si': 'a-Si',
    'a-Si/uc-Si': 'a-Si',
    'a-Si/uc-Si:H': 'a-Si',
    'a-Si/a-SiGe:H': 'a-Si',
    'a-SiGe:H': 'a-Si',
    'a-Si/ux-Si': 'a-Si',
    'a-Si/a-Si:Ge': 'a-Si',
    'a-Si/a-Si/a-Si:Ge': 'a-Si',

    'IBC': 'c-Si',
    'thin film': 'thin film',
    'bifacial': 'other',
    'Al-BSF': 'c-Si',
    'InP': 'III-V',
    'glass-backsheet': 'other',
    'other': 'other'
}
    df['pv tech'] = df['pv tech'].replace(standardized_mapping)
    df = df.set_index('paper id')

    return df

In [17]:
df_gemini_final.columns

Index(['publish year', 'doi', 'title', 'document type', 'keywords',
       'authors with affiliations', 'paper id', 'file upload time',
       'response time', 'pv technology (raw text)', 'pv tech',
       'pv technology detail', 'scope of study', 'duration (raw text)',
       'duration in years', 'start year', 'end year',
       'system capacity (raw text)', 'system capacity in watts',
       'number of pv modules in study', 'pv module nominal power (raw text)',
       'pv module nominal power in watts', 'country', 'location (raw text)',
       'location', 'pv module name', 'bifacial', 'mounting',
       'annual power degradation rate (raw text)',
       'annual power degradation rate in percent', 'location of information',
       'confidence level', 'annual degradation rate of other parameters',
       'degradation metric', 'analysis method',
       'number of measurements for degradation analysis',
       'source of initial power value', 'other examination techs',
       'faults (ra

In [18]:
df_gpt_final.columns

Index(['publish year', 'doi', 'title', 'document type', 'keywords',
       'authors with affiliations', 'paper id', 'file upload time',
       'response time', 'pv tech', 'pv tech detail', 'scope of study',
       'duration (raw text)', 'duration in years', 'start year', 'end year',
       'system capacity (raw text)', 'system capacity in watts',
       'number of pv modules in study', 'pv module nominal power (raw text)',
       'pv module nominal power in watts', 'location', 'country',
       'pv module name', 'bifacial', 'mounting',
       'annual power degradation rate (raw text)',
       'annual power degradation rate in percent', 'location of information',
       'confidence level', 'annual degradation rate of other parameters',
       'degradation metric', 'analysis method',
       'number of measurements for degradation analysis',
       'source of initial power value', 'other examination techs', 'faults',
       'grid-connected', 'materials and construction', 'note', 'paper'],

In [19]:
df_gemini_processed = rough_post_process(df_gemini_final)
df_gpt_processed = rough_post_process(df_gpt_final)

/var/folders/0j/f_d7sxyx29529cb6245wqjgm0000gq/T/ipykernel_25462/1996177106.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(values_to_replace, np.nan)
/var/folders/0j/f_d7sxyx29529cb6245wqjgm0000gq/T/ipykernel_25462/1996177106.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(values_to_replace, np.nan)


# single degradation rate match

In [23]:
aligned = df_gpt_processed[['rate']].join(
    df_gemini_processed[['rate']],
    how='inner',
    lsuffix='_gpt',
    rsuffix='_gemini'
)

rate_gpt = aligned['rate_gpt']
rate_gemini = aligned['rate_gemini']

both_nan = rate_gpt.isna() & rate_gemini.isna()
num_both_nan = both_nan.sum()

exact_same = (rate_gpt == rate_gemini) & ~both_nan
num_exact_same = exact_same.sum()

valid = ~rate_gpt.isna() & ~rate_gemini.isna()

opposite_sign_same_abs = (
    valid &
    ~exact_same &
    (np.abs(rate_gpt) == np.abs(rate_gemini)) &
    (rate_gpt * rate_gemini < 0)
)
num_opposite_sign_same_abs = opposite_sign_same_abs.sum()

percent_diff = np.abs(rate_gpt - rate_gemini) / np.abs(rate_gpt)

within_5_percent = (
    (percent_diff <= 0.05) &
    valid &
    ~exact_same &
    ~opposite_sign_same_abs
)
num_within_5_percent = within_5_percent.sum()

over_5_percent = (
    (percent_diff > 0.05) &
    valid &
    ~opposite_sign_same_abs
)
num_over_5_percent = over_5_percent.sum()

one_nan = rate_gpt.isna() ^ rate_gemini.isna()
num_one_nan = one_nan.sum()

total = len(aligned)

pct_both_nan = num_both_nan / total * 100
pct_exact_same = num_exact_same / total * 100
pct_opposite_sign_same_abs = num_opposite_sign_same_abs / total * 100
pct_within_5_percent = num_within_5_percent / total * 100
pct_over_5_percent = num_over_5_percent / total * 100
pct_one_nan = num_one_nan / total * 100

print(f"Both rate NaN: {num_both_nan} ({pct_both_nan:.2f}%)")
print(f"Rate exact same: {num_exact_same} ({pct_exact_same:.2f}%)")
print(f"Opposite sign, same |value|: {num_opposite_sign_same_abs} ({pct_opposite_sign_same_abs:.2f}%)")
print(f"Abs diff within 5%: {num_within_5_percent} ({pct_within_5_percent:.2f}%)")
print(f"Abs diff over 5%: {num_over_5_percent} ({pct_over_5_percent:.2f}%)")
print(f"One NaN, other not: {num_one_nan} ({pct_one_nan:.2f}%)")

Both rate NaN: 204 (48.23%)
Rate exact same: 52 (12.29%)
Opposite sign, same |value|: 3 (0.71%)
Abs diff within 5%: 8 (1.89%)
Abs diff over 5%: 58 (13.71%)
One NaN, other not: 98 (23.17%)


In [21]:
# 'Not match': both are not NaN, and values are different
not_match = (rate_gpt != rate_gemini) & valid

aligned[not_match]

,rate_gpt,rate_gemini
paper id,,
645,-1.5500,1.55
669,-73.0000,-4.56
775,-40.0000,-9.00
807,-1.7300,-13.50
865,-0.5725,-0.64
...,...,...
17677,-1.3000,-1.25
18061,-50.4000,-50.00
18668,-14.0000,-17.00


# combine gpt gemini results

In [ ]:
# Meta columns from source B
meta_cols = [
    'publish year', 'doi', 'title', 'document type',
    'keywords', 'authors with affiliations'
]
meta_selected = df_gemini_processed[meta_cols]

# Degradation-related columns
degr_cols = [
    'annual power degradation rate (raw text)',
    'annual power degradation rate in percent',
    'note'
]

# Short column names: A = GPT, B = Gemini
short_names_A = {
    'annual power degradation rate (raw text)': 'degr_text_A',
    'annual power degradation rate in percent': 'degr_pct_A',
    'note': 'note_A'
}

short_names_B = {
    'annual power degradation rate (raw text)': 'degr_text_B',
    'annual power degradation rate in percent': 'degr_pct_B',
    'note': 'note_B'
}

A_selected = df_gpt_processed[degr_cols].rename(columns=short_names_A)
B_selected = df_gemini_processed[degr_cols].rename(columns=short_names_B)

# Join all together
final_df = meta_selected.join(B_selected).join(A_selected)

In [36]:
gpt = pd.to_numeric(final_df['degr_pct_A'], errors='coerce')
gemini = pd.to_numeric(final_df['degr_pct_B'], errors='coerce')

df_mismatch = final_df[
    gpt.notna() &
    gemini.notna() &
    (~np.isclose(gpt, gemini, rtol=1e-6, atol=0))
]
df_mismatch = df_mismatch.reset_index(drop=False)

df_mismatch

,paper id,publish year,doi,title,document type,keywords,authors with affiliations,degr_text_B,degr_pct_B,note_B,degr_text_A,degr_pct_A,note_A
0,645,2013,10.1109/PVSC.2013.6745149,Degradation analysis of 1900 PV modules in a h...,Conference paper,NaN,"Singh J., Arizona State University, Photovolta...",0.6% and 2.5% per year,1.55,Degradation rates were calculated from namepla...,-0.6% to -2.5% per year (excluding model C4),-1.55,Modules were electrically isolated and I-V cur...
1,669,2019,10.1109/ICPES47639.2019.9105409,Power Loss Due to Photovoltaic Module Soiling ...,Conference paper,single axis tracking; soiling; solar photovolt...,"Dolan J., Atascadero High School, Atascadero, ...",inferred,-4.56,Degradation rate is inferred from the differen...,0.2% per day inferred,-73,Study measured power loss due to soiling on a ...
2,775,2015,10.1109/PVSC.2015.7356290,PV system power loss and module damage due to ...,Conference paper,bypass diodes; crystalline materials; current-...,"Brooks A.E., University of Arizona, Tucson, 85...",inferred,-9,Degradation rate is an average inferred from d...,inferred from daily energy yield loss of up to...,-40,Two types of commercial PV modules tested in T...
3,807,2019,10.1109/PVSC40753.2019.8980678,Solder Bond Degradation of Fielded PV Modules:...,Conference paper,electroluminescence imaging; fill factor; PV m...,"Sinha A., Arizona State University (ASU-PRL), ...",10-17%,-13.5,Degradation rate is inferred from the 10-17% P...,inferred,-1.73,Degradation rate was inferred from 10-year fie...
4,865,2019,10.1109/PHM-Chongqing.2018.00077,Degradation Model of Photovoltaic Modules unde...,Conference paper,Degradation model of output power; Failure phy...,"Wang R., School of Instrument Science and Opto...",0.64%,-0.64,Degradation rate is calculated using a combina...,"0.64% (normal diffusion coefficient), 0.56% (c...",-0.5725,Degradation model uses physical field simulati...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,17677,2021,10.1109/GreenTech48523.2021.00046,Residential agrivoltaics: Energy efficiency an...,Conference paper,Evapotranspiration; Photovoltaics; Power; Sens...,"Giudice B.D., George Fox University, College o...",inferred,-1.25,Degradation rate is inferred from a comparison...,"inferred, average loss about 1.3%",-1.3,Study compared ground-mount agrivoltaics syste...
65,18061,2014,10.1109/IREC.2014.6827021,Defect detection in thin-film photovoltaics; T...,Conference paper,ALD; Aluminum Oxide; Defects; Photovoltaic; WVTR,"Elrawemi M., Centre for Precicion Technologies...",inferred,-50,The degradation rate is inferred from a statem...,CIGS cell protected with PET layer lost about ...,-50.4,The study analyzed the effect of defects on wa...
66,18668,2021,10.1109/PVSC43889.2021.9518985,Snail Trail Impact on Rooftop Solar PV Plant E...,Conference paper,Field Experience; Losses; Performance; PV Modu...,"Desai A., Pandit Deendayal Energy University, ...",inferred; 17%,-17,Degradation rate is inferred from comparison o...,inferred,-14,Power generation loss due to snail trail estim...
67,19512,2018,10.1109/JPHOTOV.2017.2783853,(Al)GaInP/GaAs Tandem Solar Cells for Power Co...,Article,high temperature; III-V and concentrator photo...,"Perl E.E., National Renewable Energy Laborator...",~1%,-1,Degradation rate was determined from a 200-hou...,∼1% relative loss in efficiency after 200 h soak,-4.38,Efficiency measured before and after 200 h soa...


In [ ]:
# df_mismatch.to_pickle('df_mismatch.pkl')

# use gpt to determine the rate

In [42]:
df_mismatch = pd.read_pickle('df_mismatch.pkl')

In [ ]:
client = openai.OpenAI(
    api_key=os.environ["CBORG_API_KEY"], # Please do not store your API key in the code
    base_url="https://api.cborg.lbl.gov" # Local clients can also use https://api-local.cborg.lbl.gov
)
model = 'openai/gpt-4.1'

In [45]:
# Initialize an empty DataFrame to collect LLM results
df_llm_results = pd.DataFrame()

rows_to_analyze = [45]  # can be any list or range

for n in tqdm(rows_to_analyze):
    row = df_mismatch.iloc[n]

    tmp = row.to_frame().T.copy()
    tmp['rate_final'] = ''
    tmp['rate_reason'] = ''
    tmp['llm_success'] = np.nan

    prompt = f"""
    You are an expert in photovoltaic (PV) reliability and degradation analysis.

    Below are two independent extractions of information from the same research paper,
    produced by two different AI systems (Source A and Source B). For each source, you are given:
    - The raw text mentioning the annual power degradation rate
    - The numerical degradation rate extracted (if any; may be empty or missing)
    - Any notes or interpretation provided by the source

    Your task is to review all provided evidence and determine the most accurate
    annual power degradation rate. Then briefly explain your reasoning, including
    which source(s) you relied on and why.

    Source A:
    Raw text_A: {row.get('degr_text_A', '')}
    Extracted value_A: {row.get('degr_pct_A', '')}
    Note_A: {row.get('note_A', '')}

    Source B:
    Raw text_B: {row.get('degr_text_B', '')}
    Extracted value_B: {row.get('degr_pct_B', '')}
    Note_B: {row.get('note_B', '')}

    Respond ONLY with a Python dictionary in the following format:

    {{
        "rate_final": "<float>",
        "rate_reason": "<short explanation>"
    }}

    Important rules:
    - "rate_final" must be a single float representing the annual power degradation rate in percent.
    - Use a NEGATIVE value if performance decreases over time.
    - If a range is given (e.g., 0.5–1.0), report the mean value.
    - Do NOT include units, percent signs, or any additional text in "rate_final".
    - If no reliable degradation rate can be determined, set "rate_final" to an empty string "".
    """

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.0
        )
        res_text = response.choices[-1].message.content

        try:
            res_text_clean = re.sub(r"```json|```", "", res_text).strip()
            answer = json.loads(res_text_clean)
            llm_success = True
        except Exception:
            answer = {"rate_final": "", "rate_reason": res_text}
            llm_success = False

        tmp.at[tmp.index[0], 'rate_final'] = answer.get('rate_final', '')
        tmp.at[tmp.index[0], 'rate_reason'] = answer.get('rate_reason', '')
        tmp.at[tmp.index[0], 'llm_success'] = llm_success

    except Exception as e:
        tmp.at[tmp.index[0], 'rate_final'] = ''
        tmp.at[tmp.index[0], 'rate_reason'] = f"Error: {str(e)}"
        tmp.at[tmp.index[0], 'llm_success'] = False

    # Dynamically concatenate
    df_llm_results = pd.concat([df_llm_results, tmp], axis=0)

df_llm_results

  0%|          | 0/1 [00:00<?, ?it/s]/var/folders/0j/f_d7sxyx29529cb6245wqjgm0000gq/T/ipykernel_25462/390266331.py:70: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tmp.at[tmp.index[0], 'llm_success'] = llm_success
100%|██████████| 1/1 [00:03<00:00,  3.16s/it]


,paper id,publish year,doi,title,document type,keywords,authors with affiliations,degr_text_B,degr_pct_B,note_B,degr_text_A,degr_pct_A,note_A,rate_final,rate_reason,llm_success
45,8646,2021,10.1109/ICIEA51954.2021.9516437,Degradation Assessment of Photovoltaic Module ...,Conference paper,degradation assessment; I-V characteristic; mo...,"Zhang J., College of Mechanical and Electrical...","inferred, 4%",-4,Degradation rate is inferred from the change i...,4% in 4 years,-1,Degradation was assessed using extracted elect...,-1.0,Both sources indicate a total degradation of 4...,True
